In [1]:
import sys
sys.path.insert(0, "..")

import cv2
import numpy as np
from albumentations.augmentations.crops.transforms import CropAndPad, Crop 

from helper import create_metadata_table

In [2]:
dataset_path = "../../../Dataset/uw-madison-gi-tract-image-segmentation/"
df = create_metadata_table(dataset_path)

100%|██████████████████████████████████████████████████████████████████████████| 38496/38496 [00:16<00:00, 2362.02it/s]


In [3]:
img_path = df["img_path"].sample(1)

In [4]:
img = cv2.imread(img_path.item(), -1)
print(img.shape)

(266, 266)


In [5]:
res = CropAndPad(px=(0, 240), keep_size=False)(image=img)
print(res['image'].shape)



(654, 653)


In [6]:
res = Crop()(image=img)
print(res['image'].shape)



ValueError: Values for crop should be non negative and equal or smaller than image sizes(x_min = 0, y_min = 0, x_max = 1024, y_max = 1024, height = 266, width = 266)

In [8]:
import os
import pandas as pd
from torch.utils.data import Dataset
from helper import rle_decode

class CustomImageDataset(Dataset):
    def __init__(self, dataset_path, transform=None):
        dataset_path = "../../../Dataset/uw-madison-gi-tract-image-segmentation/"
        self.df = create_metadata_table(dataset_path)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.loc[idx, "img_path"]
        image = cv2.imread(img_path, -1)
        masks = self.df.loc[idx, ["large_bowel", "small_bowel", "stomach"]]
        
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label
    
    
    def _create_mask_array(self, masks):
        f

C:\ProgramData\Miniconda3\envs\UWMGI\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from tqdm import tqdm
for i in tqdm(range(len(df))):
    idx = 0
    img_path = df.loc[idx, "img_path"]
    masks = df.loc[idx, ["large_bowel", "small_bowel", "stomach"]]
    image = cv2.imread(img_path, -1)
    #print(image.shape)
    def _create_mask_array(masks, mask_shape):
        np_mask = list()
        for mask in masks:
            if not np.isnan(mask):
                print(idx)
                mask = rle_decode(mask, shape=mask_shape)
            else:
                mask = np.zeros(mask_shape)
            np_mask.append(mask)
        return np.stack(mask, axis=-1)

    mask = _create_mask_array(masks, mask_shape=image.shape)
    
    if mask.sum() > 10:
        print(idx)
    idx += 1


  9%|██████▌                                                                     | 3318/38496 [00:12<02:16, 258.40it/s]


KeyboardInterrupt: 

In [20]:
masks = df.loc[90, ["large_bowel", "small_bowel", "stomach"]]

In [25]:
df.loc[89, ["large_bowel", "small_bowel", "stomach"]]

large_bowel    24422 2 24684 9 24949 12 25214 14 25479 16 257...
small_bowel                                                  NaN
stomach        18511 17 18774 24 19038 30 19303 34 19568 37 1...
Name: 89, dtype: object

In [32]:
df.loc[50, ["large_bowel", "small_bowel", "stomach"]].any()

False

In [23]:
df.loc[50, ["large_bowel", "small_bowel", "stomach"]]

large_bowel    24687 4 24950 10 25215 13 25480 15 25745 17 26...
small_bowel    25662 3 25927 5 26192 6 26458 10 26723 12 2698...
stomach        18772 25 19036 32 19301 36 19566 40 19831 43 2...
Name: 90, dtype: object